https://developers.sber.ru/docs/ru/gigachat/sdk/guides/local-llms?ysclid=lvyap08s4s738058876

https://github.com/ai-forever/gigachain/blob/master/docs/docs/use_cases/question_answering/gigachat_qa.ipynb

ollama run llama3

In [13]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 KB 1.2 MB/s eta 0:00:00a 0:00:01


In [16]:
!pip install chromadb --quiet

In [1]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate

In [2]:
llm = Ollama(
    model="llama3", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

In [3]:
from langchain.schema import HumanMessage

# question = "Где Маша остановилась в Петербурге?"
# llm.invoke([HumanMessage(content=question)])

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)


In [8]:
# Для загрузки файла с текстом можно использовать команду
!wget https://stepik.org/media/attachments/lesson/1084288/The_Daughter_of_The_Commandant.pdf

--2024-05-21 20:49:49--  https://stepik.org/media/attachments/lesson/1084288/The_Daughter_of_The_Commandant.pdf
Распознаётся stepik.org (stepik.org)… 178.248.239.111
Подключение к stepik.org (stepik.org)|178.248.239.111|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 198595 (194K) [application/pdf]
Сохранение в: «The_Daughter_of_The_Commandant.pdf»

The_Daughter_of_The 100%[===================>] 193,94K  --.-KB/s    за 0,1s    

2024-05-21 20:49:50 (1,48 MB/s) - «The_Daughter_of_The_Commandant.pdf» сохранён [198595/198595]



In [4]:
import pandas as pd

df = pd.read_csv("https://stepik.org/media/attachments/lesson/1084288/pushkin_questions.csv")

In [5]:
loader = PyPDFLoader("The_Daughter_of_The_Commandant.pdf")


In [6]:
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 16


In [7]:
from chromadb.config import Settings
from langchain.vectorstores import Chroma

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cpu"}
)

In [9]:
db = Chroma.from_documents(
    documents,
    hf_embeddings_model,
    client_settings=Settings(anonymized_telemetry=False),
)

In [33]:
prompt = "Какое напутствие дал Петру отец?"
data = db.similarity_search(prompt,k=4)

In [34]:
question = f"Используя эти данные: {data}. Ответь на этот запрос  на русском языке: {prompt}"
llm.invoke([HumanMessage(content=question)])

В ответе на вопрос о напутствии, которое дал Петру отец, мы находим следующую фразу в тексте:

«отец велит беречь ему платье снову, а честь смолоду»

Таким образом, отец даст Петру напутствие: «беречь платье снову» (хранить чистоту) и «а честь смолоду» (и сохранять честь с молодых лет).

'В ответе на вопрос о напутствии, которое дал Петру отец, мы находим следующую фразу в тексте:\n\n«отец велит беречь ему платье снову, а честь смолоду»\n\nТаким образом, отец даст Петру напутствие: «беречь платье снову» (хранить чистоту) и «а честь смолоду» (и сохранять честь с молодых лет).'

In [144]:
df

,question
0,Кого убили раздетой на крыльце?
1,Где Маша остановилась в Петербурге?
2,Почему Гринев и Савельич сбились с дороги?
3,Кто издевался над Машей и морил ее голодом?
4,Куда приговорили отправить Гринева в ссылку?
5,Сколько детей было в семье Петра Гринева?
6,Кто воспитывал Петра Гринева?
7,"Сколько казаков попросил Гринев, чтобы очистит..."
8,Как далеко находилась от города Белогорская кр...
9,Как звали дочку коменданта?


In [136]:
import ollama
import chromadb


In [137]:
client = chromadb.Client()
collection = client.create_collection(name="dogm")

In [138]:

# store each document in a vector embedding database
for i, d in enumerate(documents):
    response = ollama.embeddings(model="nomic-embed-text", prompt=d.page_content)
    embedding = response["embedding"]
    print(str(i))
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        documents=[d.page_content]
    )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [152]:
# an example prompt
prompt = "Сколько детей было в семье Петра Гринева?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="nomic-embed-text"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
    n_results=4
)


In [153]:
data = results['documents']

In [151]:
# an example prompt
prompt = "Сколько детей было в семье Петра Гринева?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="nomic-embed-text"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
    n_results=4
)
data = results['documents']
question = f"Используя эти данные: {data}. Ответь на этот запрос  на русском языке: {prompt}"
llm.invoke([HumanMessage(content=question)])

В данных текста не упоминается о детях в семье Петра Гринева. Поэтому ответом будет "нет" или "не упоминалось".

'В данных текста не упоминается о детях в семье Петра Гринева. Поэтому ответом будет "нет" или "не упоминалось".'

In [148]:
question = f"Используя эти данные: {data}. Ответь на этот запрос  на русском языке: {prompt}"
llm.invoke([HumanMessage(content=question)])

Гринев и Савельич сбились с дороги из-за бурана.

'Гринев и Савельич сбились с дороги из-за бурана.'

In [143]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="llama3",
  prompt=f"Используя эти данные: {data}. Ответь на этот запрос  на русском языке: {prompt}"
)

print(output['response'])

Маша остановилась в Царском Селе.


--------------------------------------------

In [7]:
from langchain.agents import tool
from langchain.agents import initialize_agent

In [19]:
@tool
def get_basic(word: str) -> str:
    """Отвечает на вопрос, кто ты, что ты умеешь, как ты мне можешь помочь, ответ на приветствие""" # добавляем docstring
    return "Я помощник 1С"

@tool
def get_agreement(word: str) -> str:
    """Отвечает по информации по договорам и если пользователь спрашивает что-то конкретное про договора, например, текст договора, часть договора""" # добавляем docstring
    return "Вот информация по договорам"

@tool
def get_number_agreement(word: str) -> str:
    """Отвечает про количество договоров, сколько договоров в базе, показать номер договоров""" # добавляем docstring
    return "Номера заключенных договоров 1,2,3"

tools = [get_basic, get_agreement, get_number_agreement]

In [20]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True,
)

In [22]:
agent("покажи условия платежа по договору 1")



> Entering new AgentExecutor chain...
I'm ready to help!

Thought: The user wants to see the payment conditions for agreement number 1. I need to use the `get_agreement` tool to provide this information.

Action: get_agreement
Action Input: "Покажи условия платежа по договору 1I'm ready to help!

Thought: The user wants to see the payment conditions for agreement number 1. I need to use the `get_agreement` tool to provide this information.

Action: get_agreement
Action Input: "Покажи условия платежа по договору 1
Observation: Вот информация по договорам
Thought:Thought: Now that I've used the `get_agreement` action, I need to process the result to extract the payment conditions for agreement number 1.
Action: get_agreement
Action Input: "payment conditionsThought: Now that I've used the `get_agreement` action, I need to process the result to extract the payment conditions for agreement number 1.
Action: get_agreement
Action Input: "payment conditions
Observation: Вот информация по до

{'input': 'покажи условия платежа по договору 1',
 'output': 'Вот информация по договорам\n\nNote: Since `get_agreement` returns a string, I assume that it contains the payment conditions for agreement number 1. Therefore, my final answer is just a copy of the observation.'}

# Пример промта 1

In [22]:
template = '''
Определи, содержит ли сообщение в контексте предложение работы. Если содержит ответь 1, если не содержит ответь 0

Context: {context}

Question: Содержит ли сообщение предложение работы?

Answer: Верни только одно число, без текста
'''

In [23]:
prompt_template = PromptTemplate(
    input_variables=["context"],
    template=template
)

In [27]:
context = "Меня всегда интересовал вопрос безопасности"

In [28]:
prompt = prompt_template.format(context=context) # Добавляем сообщение в промпт
llm.invoke(prompt) # Ответ модели

0

'0'

# Пример промта 2

In [30]:
prompt = """Это разговор с ИИ-помощником.
Помощник обычно опирается на примеры.

Examples:
A + A = AA
B + С = BC
2 + 2 = 22

User: 1 + 1?
AI: """

print(llm.invoke(prompt))

1111


# Пример промта 3

In [12]:
llm.invoke('''Назови столицу России''')

Столица России - Москва (Moscow)

'Столица России - Москва (Moscow)'

# Пример промта 4

In [13]:
llm.invoke('''Из следующего текста извлеки информацию:

gift: Был ли товар куплен в подарок кому-то другому?
Ответь «True», если да, «False», если нет или неизвестно.

delivery_days: Сколько дней потребовалось для доставки товара? 
Если эта информация не найдена, выведи -1.

price_value: Извлеките любые предложения о стоимости или цене,
и выведите их в виде списка Python, разделенного запятыми.

text: 
Этот фен для волос просто потрясающий. Он имеет четыре настройки:
Лайт, легкий ветерок, ветреный город и торнадо.
Он прибыл через два дня, как раз к приезду моей жены -
подарок на годовщину.
Думаю, моей жене это настолько понравилось, что она потеряла дар речи.
Этот фен немного дороже, чем другие но я думаю,
что дополнительные функции того стоят.''')

Here is the extracted information:

* gift: True (the product was given as a gift to someone else)
* delivery_days: 2 (it took 2 days for the product to be delivered)
* price_value: ['Он немного дороже, чем другие'] (any mentions of cost or price)

"Here is the extracted information:\n\n* gift: True (the product was given as a gift to someone else)\n* delivery_days: 2 (it took 2 days for the product to be delivered)\n* price_value: ['Он немного дороже, чем другие'] (any mentions of cost or price)"